論文  
https://arxiv.org/abs/2204.04676<br>
GitHub<br>
https://github.com/megvii-research/NAFNet<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/NAFNet_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/megvii-research/NAFNet

## ライブラリのインストール

In [ ]:
%cd /content/NAFNet

!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

## ライブラリのインポート

In [ ]:
%cd /content/NAFNet

import gdown

import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from google.colab import files
import shutil
import glob

# 学習済みモデルのダウンロード
Access denied with the following error:<br>
が発生する場合、何回か実行

In [ ]:
%cd /content/NAFNet

# Denoise
if not os.path.exists("./experiments/pretrained_models/NAFNet-SIDD-width64.pth"):
  gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)
# Deblur
if not os.path.exists("./experiments/pretrained_models/NAFNet-REDS-width64.pth"):
  gdown.download('https://drive.google.com/uc?id=14D4V4raNYIOhETfcuuLI3bGLB-OYIv6X', "./experiments/pretrained_models/", quiet=False)
# Super Resolution
if not os.path.exists("./experiments/pretrained_models/NAFSSR-L_4x.pth"):
  gdown.download('https://drive.google.com/uc?id=1TIdQhPtBrZb2wrBdAp9l8NHINLeExOwb', "./experiments/pretrained_models/", quiet=False)


# テスト画像のセットアップ

## デモ用画像のダウンロード

In [ ]:
%cd /content/NAFNet

# Denoise
if not os.path.exists("./demo_input/noisy-demo-0.png"):
  gdown.download('https://drive.google.com/uc?id=1uKwZUgeGfBYLlPKllSuzgGUItlzb40hm', "demo_input/", quiet=False)

# Deblur
if not os.path.exists("./demo_input/blurry-reds-0.jpg"):
  gdown.download('https://drive.google.com/uc?id=1kWjrGsAvh4gOA_gn7rB9vnnQVfRINwEn', "demo_input/", quiet=False)

# Super Resolution
if not os.path.exists("./demo_input/Middlebury_lr_x4_sword2_l.png"):
  gdown.download('https://drive.google.com/uc?id=15MLvll3frPC2ICjTUnXvnLe58D_dONXc', "demo_input/", quiet=False)
if not os.path.exists("./demo_input/Middlebury_lr_x4_sword2_r.png"):
  gdown.download('https://drive.google.com/uc?id=1tedRtTen7LFHXsaC4ddwwg8ZkbEHPL0b', "demo_input/", quiet=False)

## デモ用画像のアップロード


### Denoise
[luminance noise画像](https://assets-global.website-files.com/60e4d0d0155e62117f4faef3/620c027c4aabcea88b2285d2_Example%20of%20luminance%20noise.jpg)<br>
[Chromatic noise画像](https://assets-global.website-files.com/60e4d0d0155e62117f4faef3/620c02ce030b773058eaf663_Bill-Maynard-photo-of-a-mink.jpeg)

In [ ]:
%cd /content/NAFNet

denoise_upload_folder = 'upload/denoise_input'
denoise_result_folder = 'upload/denoise_output'

if os.path.isdir(denoise_upload_folder):
    shutil.rmtree(denoise_upload_folder)
if os.path.isdir(denoise_result_folder):
    shutil.rmtree(denoise_result_folder)
os.makedirs(denoise_upload_folder)
os.makedirs(denoise_result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(denoise_upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

### Deblur
[Blur画像](https://pbblogassets.s3.amazonaws.com/uploads/2019/10/09143411/motion-blur-cover.jpg)

In [ ]:
%cd /content/NAFNet

deblur_upload_folder = 'upload/deblur_input'
deblur_result_folder = 'upload/deblur_output'

if os.path.isdir(deblur_upload_folder):
    shutil.rmtree(deblur_upload_folder)
if os.path.isdir(deblur_result_folder):
    shutil.rmtree(deblur_result_folder)
os.makedirs(deblur_upload_folder)
os.makedirs(deblur_result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(deblur_upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

### Super Resolution
[small画像](https://www.google.com/url?sa=i&url=https%3A%2F%2Fs.awa.fm%2Ftrack%2Fd9a068a1ba12b36aa983&psig=AOvVaw39U0Eb-ZfxdmYXxSnH33IX&ust=1651852662698000&source=images&cd=vfe&ved=0CAwQjRxqFwoTCMDK14HdyPcCFQAAAAAdAAAAABAD)

In [ ]:
%cd /content/NAFNet

sr_upload_folder = 'upload/sr_input'
sr_result_folder = 'upload/sr_output'

if os.path.isdir(sr_upload_folder):
    shutil.rmtree(sr_upload_folder)
if os.path.isdir(sr_result_folder):
    shutil.rmtree(sr_result_folder)
os.makedirs(sr_upload_folder)
os.makedirs(sr_result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(sr_upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

# 画像系Utility関数定義

In [ ]:
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

def sr_display(LR_l, LR_r, SR_l, SR_r):
  h,w = SR_l.shape[:2]
  LR_l = cv2.resize(LR_l, (w,h), interpolation=cv2.INTER_CUBIC)
  LR_r = cv2.resize(LR_r, (w,h), interpolation=cv2.INTER_CUBIC)
  fig = plt.figure(figsize=(w//40, h//40))
  ax1 = fig.add_subplot(2, 2, 1) 
  plt.title('Input image (Left)', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(2, 2, 2)
  plt.title('NAFSSR output (Left)', fontsize=16)
  ax2.axis('off')
  ax1.imshow(LR_l)
  ax2.imshow(SR_l)

  ax3 = fig.add_subplot(2, 2, 3) 
  plt.title('Input image (Right)', fontsize=16)
  ax3.axis('off')
  ax4 = fig.add_subplot(2, 2, 4)
  plt.title('NAFSSR output (Right)', fontsize=16)
  ax4.axis('off')
  ax3.imshow(LR_r)
  ax4.imshow(SR_r)

  plt.subplots_adjust(wspace=0.04, hspace=0.04)

def stereo_image_inference(model, img_l, img_r, save_path):
      img = torch.cat([img_l, img_r], dim=0)
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      img_L = visuals['result'][:,:3]
      img_R = visuals['result'][:,3:]
      img_L, img_R = tensor2img([img_L, img_R])
      
      imwrite(img_L, save_path.format('L'))
      imwrite(img_R, save_path.format('R'))


# Denoise

## モデルのビルド

In [ ]:
%cd /content/NAFNet

opt_path = 'options/test/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

## ダウンロード画像のDenoise

In [ ]:
%cd /content/NAFNet

input_path = 'demo_input/noisy-demo-0.png'
output_path = 'demo_output/noisy-demo-0.png'

img_input = imread(input_path)
inp = img2tensor(img_input)
single_image_inference(NAFNet, inp, output_path)
img_output = imread(output_path)
display(img_input, img_output)

## アップロード画像のDenoise

In [ ]:
%cd /content/NAFNet

input_list = sorted(glob.glob(os.path.join(denoise_upload_folder, '*')))

for input_path in input_list:
  img_input = imread(input_path)
  inp = img2tensor(img_input)
  output_path = os.path.join(denoise_result_folder, os.path.basename(input_path))
  single_image_inference(NAFNet, inp, output_path)

output_list = sorted(glob.glob(os.path.join(denoise_result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

# Deblur

## モデルのビルド

In [ ]:
%cd /content/NAFNet

opt_path = 'options/test/REDS/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

## ダウンロード画像のDeblur

In [ ]:
%cd /content/NAFNet

input_path = 'demo_input/blurry-reds-0.jpg'
output_path = 'demo_output/blurry-reds-0.jpg'

img_input = imread(input_path)
inp = img2tensor(img_input)
single_image_inference(NAFNet, inp, output_path)
img_output = imread(output_path)
display(img_input, img_output)

## アップロード画像のDeblur

In [ ]:
%cd /content/NAFNet

input_list = sorted(glob.glob(os.path.join(deblur_upload_folder, '*')))

for input_path in input_list:
  img_input = imread(input_path)
  inp = img2tensor(img_input)
  output_path = os.path.join(deblur_result_folder, os.path.basename(input_path))
  single_image_inference(NAFNet, inp, output_path)

output_list = sorted(glob.glob(os.path.join(deblur_result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

# Super Resolution

## モデルのビルド

In [ ]:
opt_path = 'options/test/NAFSSR/NAFSSR-L_4x.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFSSR = create_model(opt)

## ダウンロード画像のSR

In [ ]:
%cd /content/NAFNet

input_path_l = 'demo_input/Middlebury_lr_x4_sword2_l.png'
input_path_r = 'demo_input/Middlebury_lr_x4_sword2_r.png'
output_path = 'demo_output/Middlebury_sr_x4_sword2_{}.png'

img_l = imread(input_path_l)
inp_l = img2tensor(img_l)
img_r = imread(input_path_r)
inp_r = img2tensor(img_r)
stereo_image_inference(NAFSSR, inp_l, inp_r, output_path)
SR_l = imread(output_path.format('L'))
SR_r = imread(output_path.format('R'))
sr_display(img_l, img_r, SR_l, SR_r)

## アップロード画像のSR

In [ ]:
%cd /content/NAFNet

input_list = sorted(glob.glob(os.path.join(sr_upload_folder, '*')))

for input_path in input_list:
  img_l = imread(input_path)
  inp_l = img2tensor(img_l)
  img_r = imread(input_path)
  inp_r = img2tensor(img_r)
  stereo_image_inference(NAFSSR, inp_l, inp_r, os.path.join(sr_result_folder, "sr_result{}.png"))

output_list = sorted(glob.glob(os.path.join(sr_result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)